In [1]:
import os
import pandas as pd
import glob
import geopandas as gpd
import datetime
import numpy as np
import csv
import copy
from copy import deepcopy
import datetime
import math

# 1. Filter Weekday GPS Points

In [3]:
raw_traces = pd.read_csv(os.getcwd()+'/2020_Jan_Trace_Filtered/1_No_combined_traces.csv')

In [4]:
raw_traces['date'] = raw_traces.DATETIME.apply(lambda x:x[0:10])

In [5]:
raw_traces['date']

0           2020-02-11
1           2020-02-11
2           2020-02-11
3           2020-02-11
4           2020-02-11
               ...    
23848486    2020-01-03
23848487    2020-01-03
23848488    2020-01-03
23848489    2020-01-03
23848490    2020-01-03
Name: date, Length: 23848491, dtype: object

In [6]:
raw_traces['weekday'] = raw_traces.date.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').weekday())

In [7]:
raw_traces_weekday = raw_traces[raw_traces['weekday']<5]

In [8]:
raw_traces_weekday.to_csv(os.getcwd()+'/2020_Jan_Trace_Filtered/2_Weekday.csv', index = False)

In [9]:
raw_traces_weekday = pd.read_csv(os.getcwd()+'/2020_Jan_Trace_Filtered/2_Weekday.csv')

In [10]:
raw_traces_weekday

,ANONID,TIMESTAMP,LATITUDE,LONGITUDE,HEADING,SPEED,DATETIME,PID,point,LINKID,...,DELTA,isOD,line,DOW,HOUR,MINUTE,MINUTE_15,TOT_MIN,date,weekday
0,1302247,1581447174,38.836470,-77.091520,64.0,92.0,2020-02-11T18:52:54,19184722,POINT (-77.09152 38.83647),66362,...,3.028918,0.0,LINESTRING (-77.09735375233021 38.834660987013...,1,18,52,45,1132,2020-02-11,1
1,1302347,1581447331,38.919974,-77.108360,144.0,98.0,2020-02-11T18:55:31,19185964,POINT (-77.1083601 38.919974),41937,...,14.132664,0.0,LINESTRING (-77.13425446946015 38.935882764590...,1,18,55,45,1135,2020-02-11,1
2,1302353,1581447338,38.847782,-77.007153,0.0,65.0,2020-02-11T18:55:38,19186009,POINT (-77.007153 38.847782),12309,...,18.195850,0.0,LINESTRING (-77.00720618558938 38.845102466070...,1,18,55,45,1135,2020-02-11,1
3,1302361,1581447358,38.866260,-77.131200,88.0,64.0,2020-02-11T18:55:58,19186142,POINT (-77.13120000000001 38.86626),36890,...,0.723810,0.0,LINESTRING (-77.13416543393859 38.866162525552...,1,18,55,45,1135,2020-02-11,1
4,1302290,1581447365,38.850100,-77.007600,184.0,74.0,2020-02-11T18:56:05,19186198,POINT (-77.0076 38.8501),12311,...,14.808312,0.0,LINESTRING (-76.99971346549727 38.862991437000...,1,18,56,45,1136,2020-02-11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19705602,54206,1578061692,38.884600,-77.064190,154.0,77.0,2020-01-03T14:28:12,703238,POINT (-77.06419 38.8846),28962,...,6.605608,0.0,LINESTRING (-77.06753345782911 38.892593213960...,4,14,28,15,868,2020-01-03,4
19705603,54248,1578061692,38.916248,-77.056539,224.0,0.0,2020-01-03T14:28:12,703249,POINT (-77.05653909999999 38.9162478),47205,...,89.287024,0.0,LINESTRING (-77.05535617191158 38.915573360546...,4,14,28,15,868,2020-01-03,4
19705604,54026,1578061707,38.915711,-76.980519,215.0,0.0,2020-01-03T14:28:27,703427,POINT (-76.980519 38.915711),7491,...,67.855480,0.0,LINESTRING (-76.98154627222527 38.916883809903...,4,14,28,15,868,2020-01-03,4
19705605,54065,1578061711,38.901069,-77.038674,248.0,0.0,2020-01-03T14:28:31,703462,POINT (-77.0386735 38.9010689),47683,...,88.121942,0.0,LINESTRING (-77.03850659988062 38.901321440830...,4,14,28,15,868,2020-01-03,4


# 2.Process and Filter GPS Traces
## 2.1 Group Remaining GPS Points by ANONID

In [11]:
grouped_raw_traces = raw_traces_weekday.groupby('ANONID')

In [12]:
def get_real_time_link_pair(df):
    # results = df.groupby("LINKID").max()
    results = sorted(list(zip(df["TIMESTAMP"], df["LINKID"])), key=lambda x: x[0])
    return results

In [13]:
ID_realtraces_dataframe = grouped_raw_traces.apply(get_real_time_link_pair).reset_index()

In [14]:
ID_realtraces_dataframe = ID_realtraces_dataframe.rename(columns={0:'real_traces'})

In [15]:
ID_realtraces_dataframe

,ANONID,real_traces
0,1,"[(1577836920, 24680), (1577837041, 2666), (157..."
1,4,"[(1577836800, 3920), (1577836860, 36933), (157..."
2,6,"[(1577836800, 78050), (1577836805, 78050), (15..."
3,7,"[(1577836801, 5685), (1577836844, 5685), (1577..."
4,8,"[(1577836891, 78428), (1577836987, 8361), (157..."
...,...,...
1641737,2712947,"[(1585698997, 81838), (1585699117, 81838)]"
1641738,2712948,"[(1585699012, 83131), (1585699017, 83131), (15..."
1641739,2712949,"[(1585699017, 35957), (1585699113, 37835)]"
1641740,2712950,"[(1585699025, 59585), (1585699085, 59585), (15..."


## 2.2 Filter Traces that only contains one GPS points

In [16]:
ID_realtraces_dataframe = ID_realtraces_dataframe[ID_realtraces_dataframe['real_traces'].apply(lambda x: len(x)>1)]

In [17]:
ID_realtraces_dataframe

,ANONID,real_traces
0,1,"[(1577836920, 24680), (1577837041, 2666), (157..."
1,4,"[(1577836800, 3920), (1577836860, 36933), (157..."
2,6,"[(1577836800, 78050), (1577836805, 78050), (15..."
3,7,"[(1577836801, 5685), (1577836844, 5685), (1577..."
4,8,"[(1577836891, 78428), (1577836987, 8361), (157..."
...,...,...
1641737,2712947,"[(1585698997, 81838), (1585699117, 81838)]"
1641738,2712948,"[(1585699012, 83131), (1585699017, 83131), (15..."
1641739,2712949,"[(1585699017, 35957), (1585699113, 37835)]"
1641740,2712950,"[(1585699025, 59585), (1585699085, 59585), (15..."


## 2.3 Filter GPS traces (only keep last GPS point of a link)

In [18]:
def get_filtered_real_traces(df):
    return_list = []
    current_link = df[0][1]
    for i in range(len(df)):
        if df[i][1]!=current_link:
            return_list.append(df[i-1])
            current_link = df[i][1]
    if df[-1][1] == current_link:
        return_list.append(df[-1])
    return return_list

In [19]:
ID_realtraces_dataframe['filtered_traces'] = ID_realtraces_dataframe['real_traces'].apply(get_filtered_real_traces)

/tmp/ipykernel_26136/2945747377.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ID_realtraces_dataframe['filtered_traces'] = ID_realtraces_dataframe['real_traces'].apply(get_filtered_real_traces)


In [20]:
ID_realtraces_dataframe

,ANONID,real_traces,filtered_traces
0,1,"[(1577836920, 24680), (1577837041, 2666), (157...","[(1577836920, 24680), (1577837041, 2666), (157..."
1,4,"[(1577836800, 3920), (1577836860, 36933), (157...","[(1577836800, 3920), (1577836860, 36933), (157..."
2,6,"[(1577836800, 78050), (1577836805, 78050), (15...","[(1577836818, 78050), (1577836830, 78052), (15..."
3,7,"[(1577836801, 5685), (1577836844, 5685), (1577...","[(1577836844, 5685), (1577836892, 74841), (157..."
4,8,"[(1577836891, 78428), (1577836987, 8361), (157...","[(1577836891, 78428), (1577836987, 8361), (157..."
...,...,...,...
1641737,2712947,"[(1585698997, 81838), (1585699117, 81838)]","[(1585699117, 81838)]"
1641738,2712948,"[(1585699012, 83131), (1585699017, 83131), (15...","[(1585699027, 83131), (1585699042, 48539), (15..."
1641739,2712949,"[(1585699017, 35957), (1585699113, 37835)]","[(1585699017, 35957), (1585699113, 37835)]"
1641740,2712950,"[(1585699025, 59585), (1585699085, 59585), (15...","[(1585699176, 59585)]"


In [21]:
ID_realtraces_dataframe.loc[1, 'real_traces'] # for check purpose

[(1577836800, 3920),
 (1577836860, 36933),
 (1577836920, 83143),
 (1577836980, 83146),
 (1577837040, 83148),
 (1577837100, 15869)]

In [22]:
ID_realtraces_dataframe.loc[2, 'filtered_traces'] # for check purpose

[(1577836818, 78050),
 (1577836830, 78052),
 (1577836880, 45244),
 (1577836935, 27759),
 (1577836940, 45244),
 (1577837000, 27759),
 (1577837005, 45244),
 (1577837010, 58637),
 (1577837312, 40848),
 (1577837327, 40849)]

## 2.4 Get All Links that each Traces really Passed

In [23]:
def get_real_link(df):
    link_list = []
    for i in range(len(df)):
        link_list.append(df[i][1])
    return link_list

In [24]:
ID_realtraces_dataframe['real_link'] = ID_realtraces_dataframe['filtered_traces'].apply(get_real_link)

/tmp/ipykernel_26136/764827345.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ID_realtraces_dataframe['real_link'] = ID_realtraces_dataframe['filtered_traces'].apply(get_real_link)


In [25]:
ID_realtraces_dataframe

,ANONID,real_traces,filtered_traces,real_link
0,1,"[(1577836920, 24680), (1577837041, 2666), (157...","[(1577836920, 24680), (1577837041, 2666), (157...","[24680, 2666, 7990, 65020, 70736]"
1,4,"[(1577836800, 3920), (1577836860, 36933), (157...","[(1577836800, 3920), (1577836860, 36933), (157...","[3920, 36933, 83143, 83146, 83148, 15869]"
2,6,"[(1577836800, 78050), (1577836805, 78050), (15...","[(1577836818, 78050), (1577836830, 78052), (15...","[78050, 78052, 45244, 27759, 45244, 27759, 452..."
3,7,"[(1577836801, 5685), (1577836844, 5685), (1577...","[(1577836844, 5685), (1577836892, 74841), (157...","[5685, 74841, 48418, 39027, 676, 12311]"
4,8,"[(1577836891, 78428), (1577836987, 8361), (157...","[(1577836891, 78428), (1577836987, 8361), (157...","[78428, 8361, 8362, 55912, 17239]"
...,...,...,...,...
1641737,2712947,"[(1585698997, 81838), (1585699117, 81838)]","[(1585699117, 81838)]",[81838]
1641738,2712948,"[(1585699012, 83131), (1585699017, 83131), (15...","[(1585699027, 83131), (1585699042, 48539), (15...","[83131, 48539, 83131, 48539, 83134]"
1641739,2712949,"[(1585699017, 35957), (1585699113, 37835)]","[(1585699017, 35957), (1585699113, 37835)]","[35957, 37835]"
1641740,2712950,"[(1585699025, 59585), (1585699085, 59585), (15...","[(1585699176, 59585)]",[59585]


## 2.5 Filter Traces that only has GPS data on one Link

In [26]:
ID_realtraces_dataframe = ID_realtraces_dataframe[ID_realtraces_dataframe['real_link'].apply(lambda x: len(x)>1)]

In [27]:
ID_realtraces_dataframe

,ANONID,real_traces,filtered_traces,real_link
0,1,"[(1577836920, 24680), (1577837041, 2666), (157...","[(1577836920, 24680), (1577837041, 2666), (157...","[24680, 2666, 7990, 65020, 70736]"
1,4,"[(1577836800, 3920), (1577836860, 36933), (157...","[(1577836800, 3920), (1577836860, 36933), (157...","[3920, 36933, 83143, 83146, 83148, 15869]"
2,6,"[(1577836800, 78050), (1577836805, 78050), (15...","[(1577836818, 78050), (1577836830, 78052), (15...","[78050, 78052, 45244, 27759, 45244, 27759, 452..."
3,7,"[(1577836801, 5685), (1577836844, 5685), (1577...","[(1577836844, 5685), (1577836892, 74841), (157...","[5685, 74841, 48418, 39027, 676, 12311]"
4,8,"[(1577836891, 78428), (1577836987, 8361), (157...","[(1577836891, 78428), (1577836987, 8361), (157...","[78428, 8361, 8362, 55912, 17239]"
...,...,...,...,...
1641734,2712932,"[(1585698817, 35598), (1585698907, 75037)]","[(1585698817, 35598), (1585698907, 75037)]","[35598, 75037]"
1641736,2712946,"[(1585698984, 37935), (1585698999, 66147), (15...","[(1585698984, 37935), (1585699014, 66147), (15...","[37935, 66147, 66357, 66362, 66366, 28050, 538..."
1641738,2712948,"[(1585699012, 83131), (1585699017, 83131), (15...","[(1585699027, 83131), (1585699042, 48539), (15...","[83131, 48539, 83131, 48539, 83134]"
1641739,2712949,"[(1585699017, 35957), (1585699113, 37835)]","[(1585699017, 35957), (1585699113, 37835)]","[35957, 37835]"


## 2.6 Compute the First recording time of each GPS trace

In [28]:
def intial_time(df):
    date_time = datetime.datetime.fromtimestamp(df[0][0]).isoformat()
    return date_time[-8:]

In [29]:
ID_realtraces_dataframe['start_time'] = ID_realtraces_dataframe['real_traces'].apply(intial_time)

/tmp/ipykernel_26136/1333189269.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ID_realtraces_dataframe['start_time'] = ID_realtraces_dataframe['real_traces'].apply(intial_time)


In [30]:
ID_realtraces_dataframe

,ANONID,real_traces,filtered_traces,real_link,start_time
0,1,"[(1577836920, 24680), (1577837041, 2666), (157...","[(1577836920, 24680), (1577837041, 2666), (157...","[24680, 2666, 7990, 65020, 70736]",19:02:00
1,4,"[(1577836800, 3920), (1577836860, 36933), (157...","[(1577836800, 3920), (1577836860, 36933), (157...","[3920, 36933, 83143, 83146, 83148, 15869]",19:00:00
2,6,"[(1577836800, 78050), (1577836805, 78050), (15...","[(1577836818, 78050), (1577836830, 78052), (15...","[78050, 78052, 45244, 27759, 45244, 27759, 452...",19:00:00
3,7,"[(1577836801, 5685), (1577836844, 5685), (1577...","[(1577836844, 5685), (1577836892, 74841), (157...","[5685, 74841, 48418, 39027, 676, 12311]",19:00:01
4,8,"[(1577836891, 78428), (1577836987, 8361), (157...","[(1577836891, 78428), (1577836987, 8361), (157...","[78428, 8361, 8362, 55912, 17239]",19:01:31
...,...,...,...,...,...
1641734,2712932,"[(1585698817, 35598), (1585698907, 75037)]","[(1585698817, 35598), (1585698907, 75037)]","[35598, 75037]",19:53:37
1641736,2712946,"[(1585698984, 37935), (1585698999, 66147), (15...","[(1585698984, 37935), (1585699014, 66147), (15...","[37935, 66147, 66357, 66362, 66366, 28050, 538...",19:56:24
1641738,2712948,"[(1585699012, 83131), (1585699017, 83131), (15...","[(1585699027, 83131), (1585699042, 48539), (15...","[83131, 48539, 83131, 48539, 83134]",19:56:52
1641739,2712949,"[(1585699017, 35957), (1585699113, 37835)]","[(1585699017, 35957), (1585699113, 37835)]","[35957, 37835]",19:56:57


## 2.7 Compute the start time in minute

In [31]:
def compute_minute(df):
    minute = int(df[0:2])*60+int(df[3:5])+int(df[6:])*(1/60)
    return minute

In [32]:
ID_realtraces_dataframe['start_time_in_minute'] = ID_realtraces_dataframe['start_time'].apply(compute_minute)

/tmp/ipykernel_26136/2108949902.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ID_realtraces_dataframe['start_time_in_minute'] = ID_realtraces_dataframe['start_time'].apply(compute_minute)


In [33]:
ID_realtraces_dataframe

,ANONID,real_traces,filtered_traces,real_link,start_time,start_time_in_minute
0,1,"[(1577836920, 24680), (1577837041, 2666), (157...","[(1577836920, 24680), (1577837041, 2666), (157...","[24680, 2666, 7990, 65020, 70736]",19:02:00,1142.000000
1,4,"[(1577836800, 3920), (1577836860, 36933), (157...","[(1577836800, 3920), (1577836860, 36933), (157...","[3920, 36933, 83143, 83146, 83148, 15869]",19:00:00,1140.000000
2,6,"[(1577836800, 78050), (1577836805, 78050), (15...","[(1577836818, 78050), (1577836830, 78052), (15...","[78050, 78052, 45244, 27759, 45244, 27759, 452...",19:00:00,1140.000000
3,7,"[(1577836801, 5685), (1577836844, 5685), (1577...","[(1577836844, 5685), (1577836892, 74841), (157...","[5685, 74841, 48418, 39027, 676, 12311]",19:00:01,1140.016667
4,8,"[(1577836891, 78428), (1577836987, 8361), (157...","[(1577836891, 78428), (1577836987, 8361), (157...","[78428, 8361, 8362, 55912, 17239]",19:01:31,1141.516667
...,...,...,...,...,...,...
1641734,2712932,"[(1585698817, 35598), (1585698907, 75037)]","[(1585698817, 35598), (1585698907, 75037)]","[35598, 75037]",19:53:37,1193.616667
1641736,2712946,"[(1585698984, 37935), (1585698999, 66147), (15...","[(1585698984, 37935), (1585699014, 66147), (15...","[37935, 66147, 66357, 66362, 66366, 28050, 538...",19:56:24,1196.400000
1641738,2712948,"[(1585699012, 83131), (1585699017, 83131), (15...","[(1585699027, 83131), (1585699042, 48539), (15...","[83131, 48539, 83131, 48539, 83134]",19:56:52,1196.866667
1641739,2712949,"[(1585699017, 35957), (1585699113, 37835)]","[(1585699017, 35957), (1585699113, 37835)]","[35957, 37835]",19:56:57,1196.950000


In [30]:
Avg_Speed_Table = pd.read_csv('Link_Average_Speed.csv')

In [31]:
Avg_Speed_Table_md5 = Avg_Speed_Table.applymap(lambda x: x/5) # md5 refers to minute divide by 5, which is 12s

In [32]:
Avg_Speed_Table_md5

,63601,9456,61411,51253,17959,68552,72859,2646,70026,62617,...,40021,69229,45944,27276,13470,40554,24805,10310,12497,9951
0,0.166957,0.127059,0.072,0.181714,0.016261,0.352435,0.091765,0.229565,0.027,0.239636,...,0.193652,0.197647,0.244783,0.062069,0.044571,0.024,0.817652,0.09,0.077647,0.286107
1,0.166957,0.127059,0.072,0.181714,0.015913,0.355217,0.091765,0.229565,0.027,0.244563,...,0.195739,0.197647,0.243652,0.062069,0.044571,0.024,0.827130,0.09,0.077647,0.283442
2,0.166957,0.127059,0.072,0.181714,0.015652,0.328522,0.091765,0.229565,0.027,0.246188,...,0.195391,0.197647,0.244261,0.062069,0.044571,0.024,0.825913,0.09,0.077647,0.283519
3,0.166957,0.127059,0.072,0.181714,0.015130,0.331478,0.091765,0.229565,0.027,0.241649,...,0.192783,0.197647,0.244870,0.062069,0.044571,0.024,0.824870,0.09,0.077647,0.285475
4,0.166957,0.127059,0.072,0.181714,0.014174,0.342957,0.091765,0.229565,0.027,0.242408,...,0.193304,0.197647,0.242000,0.062069,0.044571,0.024,0.814261,0.09,0.077647,0.277230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,0.166957,0.127059,0.072,0.181714,0.016261,0.337913,0.091765,0.229565,0.027,0.236347,...,0.211043,0.197647,0.247913,0.062069,0.044571,0.024,0.814261,0.09,0.077647,0.295525
284,0.166957,0.127059,0.072,0.181714,0.013565,0.344087,0.091765,0.229565,0.027,0.240679,...,0.198696,0.197647,0.246870,0.062069,0.044571,0.024,0.816435,0.09,0.077647,0.289570
285,0.166957,0.127059,0.072,0.181714,0.016957,0.366609,0.091765,0.229565,0.027,0.242065,...,0.206696,0.197647,0.246261,0.062069,0.044571,0.024,0.824609,0.09,0.077647,0.280998
286,0.166957,0.127059,0.072,0.181714,0.016261,0.357652,0.091765,0.229565,0.027,0.243976,...,0.204957,0.197647,0.242609,0.062069,0.044571,0.024,0.820870,0.09,0.077647,0.279283


## 2.8 Connect Unlinked Link by Shortest Path (in Length)

In [34]:
########################################################################################################################
#                                               Load geo information                                                   #
########################################################################################################################
link_file = gpd.read_file('Network_consolidated/mwcog_sub_links_step6_am_clustering_internal.geojson')
node_file = gpd.read_file('Network_consolidated/mwcog_sub_nodes_step6_am_clustering_internal.geojson')
link_file.iloc[:, :-1].to_csv("links.csv")
node_file.iloc[:, :-1].to_csv("nodes.csv")
link_data = list(csv.reader(open("links.csv")))
node_data = list(csv.reader(open("nodes.csv")))
########################################################################################################################
#                                            build up node/link info dict                                              #
########################################################################################################################
Link_Dict = {}
Node_Dict = {}
for i in range(1, len(node_data)):
    Node_Dict[node_data[i][1]] = [node_data[i][2], int(node_data[i][3]), node_data[i][4], node_data[i][5]]
    # list 0 FTYPE
    # list 1 isOD
    # list 2 Longitude
    # list 3 Latitude
for j in range(1, len(link_data)):
    Link_Dict[link_data[j][1]] = [link_data[j][2], link_data[j][3], float(link_data[j][4]), link_data[j][5],
                                  link_data[j][6], link_data[j][7], int(link_data[j][8])]
    # list 0 A
    # list 1 B
    # list 2 DIST
    # list 3 POSTPD
    # list 4 FACTYPE
    # list 5 LANES
    # list 6 isOD
UpstreamLinkDict = {}
DownstreamLinkDict = {}

# build up empty dict
for i in range(1, len(node_data)):
    UpstreamLinkDict[node_data[i][1]] = []
    DownstreamLinkDict[node_data[i][1]] = []

for i in range(1, len(link_data)):
    UpstreamLinkDict[link_data[i][3]].append(link_data[i][1])
    DownstreamLinkDict[link_data[i][2]].append(link_data[i][1])

In [35]:
Maximum_Searched_Connected_Link_Number = 10
Maximum_Searched_Connected_Link_Length = 5 # in miles

def ConnectLinksByShortestLength(link, targeted_Link):
    TargetNode = Link_Dict[targeted_Link][0]
    possible_searched_list = [[link]]
    founded_list = []
    for i in range(Maximum_Searched_Connected_Link_Number):
        new_possible_searched_list = []
        for j in range(len(possible_searched_list)):
            this_link = possible_searched_list[j][-1]
            this_link_downstream_node = Link_Dict[this_link][1]
            possible_downstream_link = DownstreamLinkDict[this_link_downstream_node]
            for downstream_link in possible_downstream_link:
                Searched_List = deepcopy(possible_searched_list[j])
                Searched_List.append(str(downstream_link))
                if check_circle_for_D(Searched_List):
                    continue
                if Node_Dict[Link_Dict[downstream_link][1]][1] == 1:
                    continue
                if Link_Dict[downstream_link][1] == TargetNode:
                    founded_list.append(Searched_List)
                else:
                    new_possible_searched_list.append(Searched_List)
        possible_searched_list = new_possible_searched_list
    if len(founded_list) == 0:
        ConnectedLinkList = ['NoConnectionFounded']
    else:
        Length_list = []
        for j in range(len(founded_list)):
            length = 0
            for link_id in founded_list[j]:
                length += Link_Dict[link_id][2]
            Length_list.append(length)
        index_min = min(range(len(Length_list)), key=Length_list.__getitem__)
        if Length_list[index_min] > Maximum_Searched_Connected_Link_Length:
            ConnectedLinkList = ['NoConnectionFounded']
        else:
            ConnectedLinkList = founded_list[index_min]
            del ConnectedLinkList[0]
    return ConnectedLinkList

In [36]:
def check_circle_for_D(link_list=list):
    Indicate_number = 0
    DownStreamNode = Link_Dict[link_list[-1]][1]
    for link in link_list[:-1]:
        if DownStreamNode  == Link_Dict[link][0] or DownStreamNode == Link_Dict[link][1]:
            Indicate_number = 1
    return Indicate_number

In [37]:
def check_continuity(list): # only works for matched links
    continuity_indicator = 1
    if len(list) == 1:
        continuity_indicator = 0
    else:
        for i in range(len(list)-1):
            if Link_Dict[list[i]][1] == Link_Dict[list[i+1]][0]:
                continue
            else:
                continuity_indicator = 0
                break
    return continuity_indicator
# 1 indicates it is continuous
# 0 indicates it is not continuous

In [38]:
def match_unlinked_list(df):
    linked_list = []
    df = [str(link_int) for link_int in df] # convert int list to str list
    for i in range(len(df)):
        linked_list.append(df[i])
        if i!=len(df)-1:
            if not check_continuity([df[i],df[i+1]]):
                connected_link_list = ConnectLinksByShortestLength(df[i],df[i+1])
                linked_list.extend(connected_link_list)
    return linked_list

In [39]:
ID_realtraces_dataframe['no_OD_connected_links'] = ID_realtraces_dataframe['real_link'].apply(match_unlinked_list)

/tmp/ipykernel_29910/1214728015.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ID_realtraces_dataframe['no_OD_connected_links'] = ID_realtraces_dataframe['real_link'].apply(match_unlinked_list)


In [40]:
ID_realtraces_dataframe 

,ANONID,real_traces,filtered_traces,real_link,start_time,start_time_in_minute,no_OD_connected_links
0,1,"[(1577836920, 24680), (1577837041, 2666), (157...","[(1577836920, 24680), (1577837041, 2666), (157...","[24680, 2666, 7990, 65020, 70736]",19:02:00,1142.000000,"[24680, 73725, 26438, 81600, 81599, 26124, 119..."
1,4,"[(1577836800, 3920), (1577836860, 36933), (157...","[(1577836800, 3920), (1577836860, 36933), (157...","[3920, 36933, 83143, 83146, 83148, 15869]",19:00:00,1140.000000,"[3920, 7985, 20517, 45057, 70030, 36933, 83143..."
2,6,"[(1577836800, 78050), (1577836805, 78050), (15...","[(1577836818, 78050), (1577836830, 78052), (15...","[78050, 78052, 45244, 27759, 45244, 27759, 452...",19:00:00,1140.000000,"[78050, 78052, 73324, 23720, 45244, 27759, 452..."
3,7,"[(1577836801, 5685), (1577836844, 5685), (1577...","[(1577836844, 5685), (1577836892, 74841), (157...","[5685, 74841, 48418, 39027, 676, 12311]",19:00:01,1140.016667,"[5685, 74841, 1663, 23720, 48418, 26409, 4643,..."
4,8,"[(1577836891, 78428), (1577836987, 8361), (157...","[(1577836891, 78428), (1577836987, 8361), (157...","[78428, 8361, 8362, 55912, 17239]",19:01:31,1141.516667,"[78428, 78417, 8361, 23510, 15264, 8362, 55912..."
...,...,...,...,...,...,...,...
1641734,2712932,"[(1585698817, 35598), (1585698907, 75037)]","[(1585698817, 35598), (1585698907, 75037)]","[35598, 75037]",19:53:37,1193.616667,"[35598, 56672, 75038, 75037]"
1641736,2712946,"[(1585698984, 37935), (1585698999, 66147), (15...","[(1585698984, 37935), (1585699014, 66147), (15...","[37935, 66147, 66357, 66362, 66366, 28050, 538...",19:56:24,1196.400000,"[37935, 66147, 66357, 66362, 66366, 28050, 538..."
1641738,2712948,"[(1585699012, 83131), (1585699017, 83131), (15...","[(1585699027, 83131), (1585699042, 48539), (15...","[83131, 48539, 83131, 48539, 83134]",19:56:52,1196.866667,"[83131, 50279, 16161, 83151, 27941, 15567, 485..."
1641739,2712949,"[(1585699017, 35957), (1585699113, 37835)]","[(1585699017, 35957), (1585699113, 37835)]","[35957, 37835]",19:56:57,1196.950000,"[35957, 36934, 36930, 5378, 13209, 37835]"


### 2.8.1 Filter not Connected Traces

In [41]:
ID_realtraces_dataframe = ID_realtraces_dataframe[ID_realtraces_dataframe.apply(lambda row: 'NoConnectionFounded' not in row['no_OD_connected_links'], axis =1)]

In [42]:
ID_realtraces_dataframe

,ANONID,real_traces,filtered_traces,real_link,start_time,start_time_in_minute,no_OD_connected_links
1,4,"[(1577836800, 3920), (1577836860, 36933), (157...","[(1577836800, 3920), (1577836860, 36933), (157...","[3920, 36933, 83143, 83146, 83148, 15869]",19:00:00,1140.000000,"[3920, 7985, 20517, 45057, 70030, 36933, 83143..."
4,8,"[(1577836891, 78428), (1577836987, 8361), (157...","[(1577836891, 78428), (1577836987, 8361), (157...","[78428, 8361, 8362, 55912, 17239]",19:01:31,1141.516667,"[78428, 78417, 8361, 23510, 15264, 8362, 55912..."
5,9,"[(1577836802, 9907), (1577836837, 52969), (157...","[(1577836802, 9907), (1577836852, 52969), (157...","[9907, 52969, 78296, 78294, 15234, 26412, 3926...",19:00:02,1140.033333,"[9907, 52969, 78296, 78294, 15234, 26412, 3926..."
6,10,"[(1577836802, 17603), (1577836817, 68256), (15...","[(1577836802, 17603), (1577836817, 68256), (15...","[17603, 68256, 47976, 22155, 10107, 81098]",19:00:02,1140.033333,"[17603, 68256, 47976, 22155, 10107, 81097, 81098]"
9,17,"[(1577836807, 3567), (1577836829, 61058), (157...","[(1577836807, 3567), (1577836850, 61058), (157...","[3567, 61058, 41095, 13053, 12806, 13053, 1305...",19:00:07,1140.116667,"[3567, 70140, 61058, 41095, 13053, 12806, 1305..."
...,...,...,...,...,...,...,...
1641733,2712929,"[(1585698945, 48211), (1585699035, 28851)]","[(1585698945, 48211), (1585699035, 28851)]","[48211, 28851]",19:55:45,1195.750000,"[48211, 553, 54230, 28851]"
1641734,2712932,"[(1585698817, 35598), (1585698907, 75037)]","[(1585698817, 35598), (1585698907, 75037)]","[35598, 75037]",19:53:37,1193.616667,"[35598, 56672, 75038, 75037]"
1641736,2712946,"[(1585698984, 37935), (1585698999, 66147), (15...","[(1585698984, 37935), (1585699014, 66147), (15...","[37935, 66147, 66357, 66362, 66366, 28050, 538...",19:56:24,1196.400000,"[37935, 66147, 66357, 66362, 66366, 28050, 538..."
1641739,2712949,"[(1585699017, 35957), (1585699113, 37835)]","[(1585699017, 35957), (1585699113, 37835)]","[35957, 37835]",19:56:57,1196.950000,"[35957, 36934, 36930, 5378, 13209, 37835]"


## 2.9 Match Traces with Origin and Destination

In [43]:
########################################################################################################################
#                                            Match OD                                                                  #
########################################################################################################################
Maximum_Search_step = 10
Maximum_Search_Length = 5 # in miles

def serach_O(df):
    link = str(df[0])
    Found_Indicator = 0
    UpstreamNode = Link_Dict[link][0] # need to change if last edge in point is not edge
    if Node_Dict[UpstreamNode][1] == 1: # need to change if last edge in point is not edge
        Original_Node = UpstreamNode
        Original_Link = link
    else:
        possible_searched_list = [[link]]
        founded_list = []
        for i in range(Maximum_Search_step):
            new_possible_searched_list = []
            for j in range(len(possible_searched_list)):
                this_link = possible_searched_list[j][0]
                this_link_upstream_node = Link_Dict[this_link][0]
                possible_upstream_link = UpstreamLinkDict[this_link_upstream_node]
                for upstream_link in possible_upstream_link:
                    Searched_List = deepcopy(possible_searched_list[j])
                    Searched_List.insert(0, str(upstream_link))
                    if check_circle_for_O(Searched_List):
                        continue
                    if Node_Dict[Link_Dict[upstream_link][0]][1] == 1:
                        founded_list.append(Searched_List)
                    else:
                        new_possible_searched_list.append(Searched_List)
            possible_searched_list = new_possible_searched_list
        if len(founded_list) == 0:
            Original_Node = 'NoOriginalFounded'
        else:
            Length_list = []
            for j in range(len(founded_list)):
                length = 0
                for link_id in founded_list[j]:
                    length += Link_Dict[link_id][2]
                Length_list.append(length)
            index_min = min(range(len(Length_list)), key=Length_list.__getitem__)
            if Length_list[index_min] > Maximum_Search_Length:
                Original_Node = 'NoOriginalFounded'
            else:
                Original_Node = Link_Dict[founded_list[index_min][0]][0]
                Found_Indicator = 1
        if Found_Indicator == 1:
            Original_Link = founded_list[index_min] #[0]
        else:
            Original_Link = "NoOriginalLinkFound"
    return [Original_Node, Original_Link]


def check_circle_for_O(link_list=list):
    Indicate_number = 0
    UpStreamNode = Link_Dict[link_list[0]][0]
    for link in link_list[1:]:
        if UpStreamNode == Link_Dict[link][0] or UpStreamNode == Link_Dict[link][1]:
            Indicate_number = 1
    return Indicate_number
# 1 indicates it is a circle
# 0 indicates it is not a circle

def serach_D(df):
    Found_Indicator = 0
    link = str(df[-1])
    DownstreamNode = Link_Dict[link][1] # need to change if last edge in point is not edge
    if Node_Dict[DownstreamNode][1] == 1: # need to change if last edge in point is not edge
        Destination_Node = DownstreamNode
        Destination_Link = link
    else:
        possible_searched_list = [[link]]
        founded_list = []
        for i in range(Maximum_Search_step):
            new_possible_searched_list = []
            for j in range(len(possible_searched_list)):
                this_link = possible_searched_list[j][-1]
                this_link_downstream_node = Link_Dict[this_link][1]
                possible_downstream_link = DownstreamLinkDict[this_link_downstream_node]
                for downstream_link in possible_downstream_link:
                    Searched_List = deepcopy(possible_searched_list[j])
                    Searched_List.append(str(downstream_link))
                    if check_circle_for_D(Searched_List):
                        continue
                    if Node_Dict[Link_Dict[downstream_link][1]][1] == 1:
                        founded_list.append(Searched_List)
                    else:
                        new_possible_searched_list.append(Searched_List)
            possible_searched_list = new_possible_searched_list
        if len(founded_list) == 0:
            Destination_Node = 'NoDestinationFounded'
        else:
            Length_list = []
            for j in range(len(founded_list)):
                length = 0
                for link_id in founded_list[j]:
                    length += Link_Dict[link_id][2]
                Length_list.append(length)
            index_min = min(range(len(Length_list)), key=Length_list.__getitem__)
            if Length_list[index_min] > Maximum_Search_Length:
                Destination_Node = 'NoDestinationFounded'
            else:
                Destination_Node = Link_Dict[founded_list[index_min][-1]][1]
                Found_Indicator= 1
        if Found_Indicator == 1:
            Destination_Link = founded_list[index_min] #[-1]
        else:
            Destination_Link = "NoDestinationLinkFound"

    return [Destination_Node, Destination_Link]

def check_circle_for_D(link_list=list):
    Indicate_number = 0
    DownStreamNode = Link_Dict[link_list[-1]][1]
    for link in link_list[:-1]:
        if DownStreamNode  == Link_Dict[link][0] or DownStreamNode == Link_Dict[link][1]:
            Indicate_number = 1
    return Indicate_number
# 1 indicates it is a circle
# 0 indicates it is not a circle

In [44]:
ID_realtraces_dataframe['Origin_Info'] = ID_realtraces_dataframe['real_link'].apply(serach_O)

/tmp/ipykernel_29910/2251197422.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ID_realtraces_dataframe['Origin_Info'] = ID_realtraces_dataframe['real_link'].apply(serach_O)


In [45]:
ID_realtraces_dataframe['Origin_Node'] = ID_realtraces_dataframe['Origin_Info'].apply(lambda x:x[0])

/tmp/ipykernel_29910/1360004384.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ID_realtraces_dataframe['Origin_Node'] = ID_realtraces_dataframe['Origin_Info'].apply(lambda x:x[0])


In [46]:
ID_realtraces_dataframe['Origin_Link'] = ID_realtraces_dataframe['Origin_Info'].apply(lambda x:x[1])

/tmp/ipykernel_29910/859108360.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ID_realtraces_dataframe['Origin_Link'] = ID_realtraces_dataframe['Origin_Info'].apply(lambda x:x[1])


In [47]:
ID_realtraces_dataframe['Destination_Info'] = ID_realtraces_dataframe['real_link'].apply(serach_D)

/tmp/ipykernel_29910/2636614766.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ID_realtraces_dataframe['Destination_Info'] = ID_realtraces_dataframe['real_link'].apply(serach_D)


In [48]:
ID_realtraces_dataframe['Destination_Node'] = ID_realtraces_dataframe['Destination_Info'].apply(lambda x:x[0])

/tmp/ipykernel_29910/1496685939.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ID_realtraces_dataframe['Destination_Node'] = ID_realtraces_dataframe['Destination_Info'].apply(lambda x:x[0])


In [49]:
ID_realtraces_dataframe['Destination_Link'] = ID_realtraces_dataframe['Destination_Info'].apply(lambda x:x[1])

/tmp/ipykernel_29910/3968118867.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ID_realtraces_dataframe['Destination_Link'] = ID_realtraces_dataframe['Destination_Info'].apply(lambda x:x[1])


In [52]:
ID_realtraces_dataframe

,ANONID,real_traces,filtered_traces,real_link,start_time,start_time_in_minute,no_OD_connected_links,Origin_Info,Origin_Node,Origin_Link,Destination_Info,Destination_Node,Destination_Link
1,4,"[(1577836800, 3920), (1577836860, 36933), (157...","[(1577836800, 3920), (1577836860, 36933), (157...","[3920, 36933, 83143, 83146, 83148, 15869]",19:00:00,1140.000000,"[3920, 7985, 20517, 45057, 70030, 36933, 83143...","[100318, [34866, 7071, 70635, 23516, 3920]]",100318,"[34866, 7071, 70635, 23516, 3920]","[100040, [15869, 45620, 34865, 74832]]",100040,"[15869, 45620, 34865, 74832]"
4,8,"[(1577836891, 78428), (1577836987, 8361), (157...","[(1577836891, 78428), (1577836987, 8361), (157...","[78428, 8361, 8362, 55912, 17239]",19:01:31,1141.516667,"[78428, 78417, 8361, 23510, 15264, 8362, 55912...","[100258, [78436, 78433, 78428]]",100258,"[78436, 78433, 78428]","[100161, [17239, 35968, 12924]]",100161,"[17239, 35968, 12924]"
5,9,"[(1577836802, 9907), (1577836837, 52969), (157...","[(1577836802, 9907), (1577836852, 52969), (157...","[9907, 52969, 78296, 78294, 15234, 26412, 3926...",19:00:02,1140.033333,"[9907, 52969, 78296, 78294, 15234, 26412, 3926...","[100099, [59551, 37055, 38670, 9907]]",100099,"[59551, 37055, 38670, 9907]","[100211, [60357, 60359, 60358, 56893]]",100211,"[60357, 60359, 60358, 56893]"
6,10,"[(1577836802, 17603), (1577836817, 68256), (15...","[(1577836802, 17603), (1577836817, 68256), (15...","[17603, 68256, 47976, 22155, 10107, 81098]",19:00:02,1140.033333,"[17603, 68256, 47976, 22155, 10107, 81097, 81098]","[100161, [12930, 72608, 51056, 17603]]",100161,"[12930, 72608, 51056, 17603]","[100091, [81098, 15599]]",100091,"[81098, 15599]"
9,17,"[(1577836807, 3567), (1577836829, 61058), (157...","[(1577836807, 3567), (1577836850, 61058), (157...","[3567, 61058, 41095, 13053, 12806, 13053, 1305...",19:00:07,1140.116667,"[3567, 70140, 61058, 41095, 13053, 12806, 1305...","[100190, [50668, 70865, 69848, 3567]]",100190,"[50668, 70865, 69848, 3567]","[100050, [75817, 19137]]",100050,"[75817, 19137]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1641733,2712929,"[(1585698945, 48211), (1585699035, 28851)]","[(1585698945, 48211), (1585699035, 28851)]","[48211, 28851]",19:55:45,1195.750000,"[48211, 553, 54230, 28851]","[100304, [2584, 48211]]",100304,"[2584, 48211]","[100060, [28851, 54235, 66361, 52094, 33358]]",100060,"[28851, 54235, 66361, 52094, 33358]"
1641734,2712932,"[(1585698817, 35598), (1585698907, 75037)]","[(1585698817, 35598), (1585698907, 75037)]","[35598, 75037]",19:53:37,1193.616667,"[35598, 56672, 75038, 75037]","[100264, [74768, 10156, 35598]]",100264,"[74768, 10156, 35598]","[100003, [75037, 27651, 54265]]",100003,"[75037, 27651, 54265]"
1641736,2712946,"[(1585698984, 37935), (1585698999, 66147), (15...","[(1585698984, 37935), (1585699014, 66147), (15...","[37935, 66147, 66357, 66362, 66366, 28050, 538...",19:56:24,1196.400000,"[37935, 66147, 66357, 66362, 66366, 28050, 538...","[100021, [27664, 17698, 18094, 77144, 37935]]",100021,"[27664, 17698, 18094, 77144, 37935]","[100295, [11710, 51284]]",100295,"[11710, 51284]"
1641739,2712949,"[(1585699017, 35957), (1585699113, 37835)]","[(1585699017, 35957), (1585699113, 37835)]","[35957, 37835]",19:56:57,1196.950000,"[35957, 36934, 36930, 5378, 13209, 37835]","[100161, [12923, 9962, 36932, 35957]]",100161,"[12923, 9962, 36932, 35957]","[100318, [37835, 1655, 13202]]",100318,"[37835, 1655, 13202]"


In [51]:
ID_realtraces_dataframe.to_csv('OD_connceted_Traces', index=False) # Save for Safe

### 2.9.1 Filter Traces that not matched Origins and Destinations

In [64]:
ID_realtraces_dataframe = ID_realtraces_dataframe[ID_realtraces_dataframe.apply(lambda row: row['Origin_Node']!='NoOriginalFounded', axis =1)]
ID_realtraces_dataframe.shape[0]

1177032

In [58]:
ID_realtraces_dataframe = ID_realtraces_dataframe[ID_realtraces_dataframe.apply(lambda row: row['Origin_Link']!="NoOriginalLinkFound", axis =1)]
ID_realtraces_dataframe.shape[0]

In [65]:
ID_realtraces_dataframe = ID_realtraces_dataframe[ID_realtraces_dataframe.apply(lambda row: row['Destination_Node']!='NoDestinationFounded', axis =1)]
ID_realtraces_dataframe.shape[0]

1176999

In [66]:
ID_realtraces_dataframe = ID_realtraces_dataframe[ID_realtraces_dataframe.apply(lambda row: row['Destination_Link']!="NoDestinationLinkFound", axis =1)]
ID_realtraces_dataframe.shape[0]

1176999

### 2.9.2 Filter Traces has same Origins and Destinations 
### (Short Traces not Necessary for Analysis)

In [68]:
ID_realtraces_dataframe = ID_realtraces_dataframe[ID_realtraces_dataframe.apply(lambda row: row['Destination_Node']!=row['Origin_Node'], axis =1)]
ID_realtraces_dataframe.shape[0]

1046897

## 2.10 Concate all conected traces

In [69]:
ID_realtraces_dataframe['OD_connected_link'] = ID_realtraces_dataframe.apply(lambda row: row['Origin_Link'][:-1]+row['no_OD_connected_links']+row['Destination_Link'][1:], axis =1)

In [71]:
del ID_realtraces_dataframe['Destination_Info']
del ID_realtraces_dataframe['Origin_Info']

In [72]:
ID_realtraces_dataframe

,ANONID,real_traces,filtered_traces,real_link,start_time,start_time_in_minute,no_OD_connected_links,Origin_Node,Origin_Link,Destination_Node,Destination_Link,OD_connected_link
1,4,"[(1577836800, 3920), (1577836860, 36933), (157...","[(1577836800, 3920), (1577836860, 36933), (157...","[3920, 36933, 83143, 83146, 83148, 15869]",19:00:00,1140.000000,"[3920, 7985, 20517, 45057, 70030, 36933, 83143...",100318,"[34866, 7071, 70635, 23516, 3920]",100040,"[15869, 45620, 34865, 74832]","[34866, 7071, 70635, 23516, 3920, 7985, 20517,..."
4,8,"[(1577836891, 78428), (1577836987, 8361), (157...","[(1577836891, 78428), (1577836987, 8361), (157...","[78428, 8361, 8362, 55912, 17239]",19:01:31,1141.516667,"[78428, 78417, 8361, 23510, 15264, 8362, 55912...",100258,"[78436, 78433, 78428]",100161,"[17239, 35968, 12924]","[78436, 78433, 78428, 78417, 8361, 23510, 1526..."
5,9,"[(1577836802, 9907), (1577836837, 52969), (157...","[(1577836802, 9907), (1577836852, 52969), (157...","[9907, 52969, 78296, 78294, 15234, 26412, 3926...",19:00:02,1140.033333,"[9907, 52969, 78296, 78294, 15234, 26412, 3926...",100099,"[59551, 37055, 38670, 9907]",100211,"[60357, 60359, 60358, 56893]","[59551, 37055, 38670, 9907, 52969, 78296, 7829..."
6,10,"[(1577836802, 17603), (1577836817, 68256), (15...","[(1577836802, 17603), (1577836817, 68256), (15...","[17603, 68256, 47976, 22155, 10107, 81098]",19:00:02,1140.033333,"[17603, 68256, 47976, 22155, 10107, 81097, 81098]",100161,"[12930, 72608, 51056, 17603]",100091,"[81098, 15599]","[12930, 72608, 51056, 17603, 68256, 47976, 221..."
9,17,"[(1577836807, 3567), (1577836829, 61058), (157...","[(1577836807, 3567), (1577836850, 61058), (157...","[3567, 61058, 41095, 13053, 12806, 13053, 1305...",19:00:07,1140.116667,"[3567, 70140, 61058, 41095, 13053, 12806, 1305...",100190,"[50668, 70865, 69848, 3567]",100050,"[75817, 19137]","[50668, 70865, 69848, 3567, 70140, 61058, 4109..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1641733,2712929,"[(1585698945, 48211), (1585699035, 28851)]","[(1585698945, 48211), (1585699035, 28851)]","[48211, 28851]",19:55:45,1195.750000,"[48211, 553, 54230, 28851]",100304,"[2584, 48211]",100060,"[28851, 54235, 66361, 52094, 33358]","[2584, 48211, 553, 54230, 28851, 54235, 66361,..."
1641734,2712932,"[(1585698817, 35598), (1585698907, 75037)]","[(1585698817, 35598), (1585698907, 75037)]","[35598, 75037]",19:53:37,1193.616667,"[35598, 56672, 75038, 75037]",100264,"[74768, 10156, 35598]",100003,"[75037, 27651, 54265]","[74768, 10156, 35598, 56672, 75038, 75037, 276..."
1641736,2712946,"[(1585698984, 37935), (1585698999, 66147), (15...","[(1585698984, 37935), (1585699014, 66147), (15...","[37935, 66147, 66357, 66362, 66366, 28050, 538...",19:56:24,1196.400000,"[37935, 66147, 66357, 66362, 66366, 28050, 538...",100021,"[27664, 17698, 18094, 77144, 37935]",100295,"[11710, 51284]","[27664, 17698, 18094, 77144, 37935, 66147, 663..."
1641739,2712949,"[(1585699017, 35957), (1585699113, 37835)]","[(1585699017, 35957), (1585699113, 37835)]","[35957, 37835]",19:56:57,1196.950000,"[35957, 36934, 36930, 5378, 13209, 37835]",100161,"[12923, 9962, 36932, 35957]",100318,"[37835, 1655, 13202]","[12923, 9962, 36932, 35957, 36934, 36930, 5378..."


## 2.11 Check if there is a circile in OD_Connected_Link

In [76]:
ID_realtraces_dataframe['if_circle'] = ID_realtraces_dataframe['OD_connected_link'].apply(check_circle_for_D)

#### zero implies good

In [79]:
ID_realtraces_dataframe['if_circle'].sum()

0

In [80]:
# save for safe
ID_realtraces_dataframe.to_csv(os.getcwd()+'/2020_JanFeb_Trace_Filtered/3_OD_connected_traces.csv', index=False)

In [96]:
del ID_realtraces_dataframe['if_circle']

# 3. Group Traces by OD

In [97]:
ID_realtraces_dataframe_grouped = ID_realtraces_dataframe.groupby(['Origin_Node','Destination_Node'])

## 3.1 Count the number of traces in each OD pair

In [98]:
ID_realtraces_dataframe_grouped_size = ID_realtraces_dataframe_grouped.size()

In [99]:
ID_realtraces_dataframe_grouped_size = ID_realtraces_dataframe_grouped_size.reset_index(name='size')

## 3.2 Compute the distance between ODs

In [120]:
def haversine(Node1, Node2):

    R = 3958.8
    lat1 = float(Node_Dict[Node1][3])
    lon1 = float(Node_Dict[Node1][2])
    lat2 = float(Node_Dict[Node2][3])
    lon2 = float(Node_Dict[Node2][2])

    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    
    return R * c # in miles

In [122]:
ID_realtraces_dataframe_grouped_size['distance'] = ID_realtraces_dataframe_grouped_size.apply(lambda row: haversine(row['Origin_Node'], row['Destination_Node']),  axis =1)

In [123]:
ID_realtraces_dataframe_grouped_size

,Origin_Node,Destination_Node,size,distance
0,100002,100004,1,6.054787
1,100002,100005,5,2.673413
2,100002,100008,29,1.062729
3,100002,100017,1,5.982358
4,100002,100018,15,4.926482
...,...,...,...,...
16498,100397,100365,7,2.707147
16499,100397,100370,5,2.246205
16500,100397,100372,37,6.327224
16501,100397,100379,2,3.059361


In [124]:
ID_realtraces_dataframe_grouped_size.to_csv(os.getcwd()+'/2020_Jan_Trace_Filtered/OD_trace_size.csv', index=False)

## 3.3 Group Trace in same OD into seperate csv files

In [101]:
for name, group in ID_realtraces_dataframe_grouped:
    orgin_node = int(name[0])
    destination_node = int(name[1])
    group.to_csv(os.getcwd()+f'/2020_Jan_Trace_Filtered/Jan_OD_Traces/{orgin_node}_{destination_node}.csv', index=False)

## 3.4 Select OD pair by trace number and distane

In [126]:
minimum_trace_number = 10
minimum_distance = 3

In [127]:
ID_realtraces_dataframe_selected_OD = ID_realtraces_dataframe_grouped_size[(ID_realtraces_dataframe_grouped_size['size']>=minimum_trace_number)&(ID_realtraces_dataframe_grouped_size['distance']>=minimum_distance)]

In [128]:
ID_realtraces_dataframe_selected_OD

,Origin_Node,Destination_Node,size,distance
4,100002,100018,15,4.926482
6,100002,100022,10,3.207561
43,100002,100181,12,3.937510
50,100002,100223,32,3.158921
52,100002,100231,20,3.075293
...,...,...,...,...
16411,100396,100337,84,3.044628
16416,100396,100350,25,3.429734
16417,100396,100351,28,4.230645
16424,100396,100372,37,7.950986


In [129]:
ID_realtraces_dataframe_selected_OD['size'].sum()

217664

In [130]:
ID_realtraces_dataframe_selected_OD.to_csv(os.getcwd()+'/2020_Jan_Trace_Filtered/selected_OD_trace_size.csv', index=False)

## 3.4 FIlter Traces by Selected OD Pair

In [138]:
ID_realtraces_dataframe_selected_OD['OD_pair'] = ID_realtraces_dataframe_selected_OD.apply(lambda row: [row['Origin_Node'], row['Destination_Node']], axis =1)

/tmp/ipykernel_29910/1089033949.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ID_realtraces_dataframe_selected_OD['OD_pair'] = ID_realtraces_dataframe_selected_OD.apply(lambda row: [row['Origin_Node'], row['Destination_Node']], axis =1)


In [139]:
selected_OD_pair_list = list(ID_realtraces_dataframe_selected_OD['OD_pair'])

In [140]:
del ID_realtraces_dataframe_selected_OD['OD_pair']

In [142]:
selected_OD_pair_list

[['100002', '100018'],
 ['100002', '100022'],
 ['100002', '100181'],
 ['100002', '100223'],
 ['100002', '100231'],
 ['100002', '100283'],
 ['100002', '100351'],
 ['100002', '100372'],
 ['100003', '100099'],
 ['100003', '100161'],
 ['100003', '100183'],
 ['100003', '100190'],
 ['100003', '100206'],
 ['100003', '100242'],
 ['100003', '100253'],
 ['100003', '100254'],
 ['100003', '100291'],
 ['100003', '100297'],
 ['100003', '100321'],
 ['100003', '100349'],
 ['100004', '100002'],
 ['100004', '100003'],
 ['100004', '100005'],
 ['100004', '100009'],
 ['100004', '100017'],
 ['100004', '100018'],
 ['100004', '100022'],
 ['100004', '100023'],
 ['100004', '100027'],
 ['100004', '100048'],
 ['100004', '100050'],
 ['100004', '100056'],
 ['100004', '100060'],
 ['100004', '100062'],
 ['100004', '100064'],
 ['100004', '100066'],
 ['100004', '100075'],
 ['100004', '100079'],
 ['100004', '100081'],
 ['100004', '100092'],
 ['100004', '100095'],
 ['100004', '100099'],
 ['100004', '100116'],
 ['100004',

In [136]:
ID_realtraces_dataframe['OD_pair'] = ID_realtraces_dataframe.apply(lambda row: [row['Origin_Node'], row['Destination_Node']], axis =1)

In [144]:
selected_ID_realtraces_dataframe = ID_realtraces_dataframe[ID_realtraces_dataframe.apply(lambda row: row['OD_pair'] in selected_OD_pair_list, axis =1)]

In [146]:
selected_ID_realtraces_dataframe.to_csv(os.getcwd()+'/2020_Jan_Trace_Filtered/4_selected_OD_connected_traces.csv', index=False)

## 3.5 Group Selected Trace in same OD into seperate csv files

In [147]:
selected_ID_realtraces_dataframe_grouped = selected_ID_realtraces_dataframe.groupby(['Origin_Node','Destination_Node'])

In [148]:
for name, group in selected_ID_realtraces_dataframe_grouped:
    orgin_node = int(name[0])
    destination_node = int(name[1])
    group.to_csv(os.getcwd()+f'/2020_Jan_Trace_Filtered/Selected_Jan_OD_Traces/{orgin_node}_{destination_node}.csv', index=False)